firebase rpc wrapper?


In [ ]:
var cors = require('cors')({origin: true});
var importer = require('../Core');
var process = require('process');
var getResult = importer.import('rpc result');
var functions = require('firebase-functions'),
    admin = require('firebase-admin');

try {
    admin.initializeApp(functions.config().firebase);
} catch (e) {
    console.log(e);
}

if(typeof module.exports === 'undefined') {
    module.exports = {};
}

module.exports.rpc = functions.https.onRequest((req, res) => {
    process.env.HOME = process.env.HOMEPATH = process.env.USERPROFILE = '';
    const config = functions.config();
    const services = Object.keys(config).reduce(k => {
        Object.keys(config[k]).forEach(s => {
            process.env[k.toUpperCase() + '_' + s.toUpperCase()] = config[k][s];
        });
    });
    
    cors(req, res, () => {
        return Promise.resolve([])
            .then(() => getResult({
                command: req.body['function'] || req.query['function'],
                result: importer.interpret(req.body['function'] || req.query['function']),
                body: req.method === 'POST' ? req.body : req.query,
                circles: ['Public']
            }))
            .then(r => res.status(200).send(r))
            // TODO: object assign error?
            .catch(e => { 
                console.error(e);
                return res.status(500).send(Object.getOwnPropertyNames(e).reduce((alt, key) => {
                    alt[key] = e[key];
                    return alt;
                }, {}));
            });
    });
})
